In [1]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler


In [2]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # In this example, we don't change the model architecture
        # due to simplicity.
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


In [3]:
# Change these values if you want the training to run quicker or slower.
EPOCH_SIZE = 512
TEST_SIZE = 256


def train(model, optimizer, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total


In [4]:
def train_mnist(config):
    # Data Setup
    mnist_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

    train_loader = DataLoader(
        datasets.MNIST("~/data", train=True, download=True, transform=mnist_transforms), batch_size=64, shuffle=True
    )
    test_loader = DataLoader(
        datasets.MNIST("~/data", train=False, transform=mnist_transforms), batch_size=64, shuffle=True
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ConvNet()
    model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=config["momentum"])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)

        # Send the current training result back to Tune
        tune.report(mean_accuracy=acc)

        if i % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")


In [5]:
search_space = {
    "lr": tune.sample_from(lambda spec: 10 ** (-10 * np.random.rand())),
    "momentum": tune.uniform(0.1, 0.9),
}

# Uncomment this to enable distributed execution
# `ray.init(address="auto")`

# Download the dataset first
datasets.MNIST("~/data", train=True, download=True)

analysis = tune.run(
    train_mnist,
    config=search_space,
    local_dir="./ray_train_results",
    resources_per_trial={"gpu": 1},
)


2022-06-07 22:49:27,026	WARNING function_runner.py:598 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-06-07 22:49:27,229	INFO trial_runner.py:803 -- starting train_mnist_07581_00000
2022-06-07 22:49:27,276	ERROR syncer.py:119 -- Log sync requires rsync to be installed.


Trial name,status,loc,lr,momentum
train_mnist_07581_00000,RUNNING,127.0.0.1:34692,1.24144e-10,0.361751


Result for train_mnist_07581_00000:
  date: 2022-06-07_22-49-30
  done: false
  experiment_id: 4e430133e6c84992962738bcec9191fa
  hostname: DESKTOP-1OR65MK
  iterations_since_restore: 1
  mean_accuracy: 0.134375
  node_ip: 127.0.0.1
  pid: 34692
  time_since_restore: 0.24604201316833496
  time_this_iter_s: 0.24604201316833496
  time_total_s: 0.24604201316833496
  timestamp: 1654613370
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 07581_00000
  warmup_time: 0.003958702087402344
  


Trial name,status,loc,lr,momentum,acc,iter,total time (s)
train_mnist_07581_00000,RUNNING,127.0.0.1:34692,1.24144e-10,0.361751,0.1,10,2.20203


Result for train_mnist_07581_00000:
  date: 2022-06-07_22-49-32
  done: true
  experiment_id: 4e430133e6c84992962738bcec9191fa
  experiment_tag: 0_lr=1.2414e-10,momentum=0.36175
  hostname: DESKTOP-1OR65MK
  iterations_since_restore: 10
  mean_accuracy: 0.1
  node_ip: 127.0.0.1
  pid: 34692
  time_since_restore: 2.2020349502563477
  time_this_iter_s: 0.20333480834960938
  time_total_s: 2.2020349502563477
  timestamp: 1654613372
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 07581_00000
  warmup_time: 0.003958702087402344
  


Trial name,status,loc,lr,momentum,acc,iter,total time (s)
train_mnist_07581_00000,TERMINATED,127.0.0.1:34692,1.24144e-10,0.361751,0.1,10,2.20203


(pid=) 2022-06-07 22:49:32,508	INFO context.py:67 -- Exec'ing worker with command: "c:\Anaconda3\envs\transformers\python.exe" c:\Anaconda3\envs\transformers\lib\site-packages\ray\workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=1306 --object-store-name=tcp://127.0.0.1:58547 --raylet-name=tcp://127.0.0.1:34295 --redis-address=None --storage=None --temp-dir=C:\Users\tzh\AppData\Local\Temp\ray --metrics-agent-port=25171 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:19312 --redis-password=5241590000000000 --startup-token=12 --runtime-env-hash=-1375307434
2022-06-07 22:49:32,573	INFO tune.py:701 -- Total run time: 5.56 seconds (5.35 seconds for the tuning loop).


In [6]:
dfs = analysis.trial_dataframes
# [d.mean_accuracy.plot() for d in dfs.values()]


In [7]:
df = list(dfs.values())[0]
df


,mean_accuracy,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,warmup_time
0,0.134375,0.246042,False,NaN,NaN,1,07581_00000,4e430133e6c84992962738bcec9191fa,2022-06-07_22-49-30,1654613370,0.246042,34692,DESKTOP-1OR65MK,127.0.0.1,0.246042,0,1,0.003959
1,0.106250,0.232400,False,NaN,NaN,2,07581_00000,4e430133e6c84992962738bcec9191fa,2022-06-07_22-49-30,1654613370,0.478442,34692,DESKTOP-1OR65MK,127.0.0.1,0.478442,0,2,0.003959
2,0.096875,0.212301,False,NaN,NaN,3,07581_00000,4e430133e6c84992962738bcec9191fa,2022-06-07_22-49-30,1654613370,0.690742,34692,DESKTOP-1OR65MK,127.0.0.1,0.690742,0,3,0.003959
3,0.131250,0.250243,False,NaN,NaN,4,07581_00000,4e430133e6c84992962738bcec9191fa,2022-06-07_22-49-31,1654613371,0.940985,34692,DESKTOP-1OR65MK,127.0.0.1,0.940985,0,4,0.003959
4,0.075000,0.215931,False,NaN,NaN,5,07581_00000,4e430133e6c84992962738bcec9191fa,2022-06-07_22-49-31,1654613371,1.156916,34692,DESKTOP-1OR65MK,127.0.0.1,1.156916,0,5,0.003959
5,0.112500,0.205211,False,NaN,NaN,6,07581_00000,4e430133e6c84992962738bcec9191fa,2022-06-07_22-49-31,1654613371,1.362128,34692,DESKTOP-1OR65MK,127.0.0.1,1.362128,0,6,0.003959
6,0.137500,0.211543,False,NaN,NaN,7,07581_00000,4e430133e6c84992962738bcec9191fa,2022-06-07_22-49-31,1654613371,1.573671,34692,DESKTOP-1OR65MK,127.0.0.1,1.573671,0,7,0.003959
7,0.115625,0.200710,False,NaN,NaN,8,07581_00000,4e430133e6c84992962738bcec9191fa,2022-06-07_22-49-32,1654613372,1.774381,34692,DESKTOP-1OR65MK,127.0.0.1,1.774381,0,8,0.003959
8,0.125000,0.224319,False,NaN,NaN,9,07581_00000,4e430133e6c84992962738bcec9191fa,2022-06-07_22-49-32,1654613372,1.998700,34692,DESKTOP-1OR65MK,127.0.0.1,1.998700,0,9,0.003959
9,0.100000,0.203335,False,NaN,NaN,10,07581_00000,4e430133e6c84992962738bcec9191fa,2022-06-07_22-49-32,1654613372,2.202035,34692,DESKTOP-1OR65MK,127.0.0.1,2.202035,0,10,0.003959


In [8]:
analysis = tune.run(
    train_mnist,
    num_samples=20,
    scheduler=ASHAScheduler(metric="mean_accuracy", mode="max"),
    config=search_space,
    local_dir="./ray_train_results",
    resources_per_trial={"gpu": 1},
)

# Obtain a trial dataframe from all run trials of this `tune.run` call.
dfs = analysis.trial_dataframes


2022-06-07 22:49:32,919	INFO trial_runner.py:803 -- starting train_mnist_0abb8_00000


Trial name,status,loc,lr,momentum
train_mnist_0abb8_00000,RUNNING,127.0.0.1:28424,0.00715404,0.439615
train_mnist_0abb8_00001,PENDING,,0.00230887,0.681205
train_mnist_0abb8_00002,PENDING,,1.44449e-08,0.681438
train_mnist_0abb8_00003,PENDING,,2.5637e-08,0.606991
train_mnist_0abb8_00004,PENDING,,7.35795e-07,0.710213
train_mnist_0abb8_00005,PENDING,,0.000168464,0.310138
train_mnist_0abb8_00006,PENDING,,0.429744,0.144786
train_mnist_0abb8_00007,PENDING,,0.000358131,0.690782
train_mnist_0abb8_00008,PENDING,,9.75779e-10,0.227704
train_mnist_0abb8_00009,PENDING,,2.36833e-07,0.257793


Result for train_mnist_0abb8_00000:
  date: 2022-06-07_22-49-36
  done: false
  experiment_id: bb3a058f213f4036827c15c4f63fc729
  hostname: DESKTOP-1OR65MK
  iterations_since_restore: 1
  mean_accuracy: 0.08125
  node_ip: 127.0.0.1
  pid: 28424
  time_since_restore: 0.25426816940307617
  time_this_iter_s: 0.25426816940307617
  time_total_s: 0.25426816940307617
  timestamp: 1654613376
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 0abb8_00000
  warmup_time: 0.004000425338745117
  


Trial name,status,loc,lr,momentum,acc,iter,total time (s)
train_mnist_0abb8_00000,RUNNING,127.0.0.1:28424,0.00715404,0.439615,0.74375,10,2.26315
train_mnist_0abb8_00001,PENDING,,0.00230887,0.681205,,,
train_mnist_0abb8_00002,PENDING,,1.44449e-08,0.681438,,,
train_mnist_0abb8_00003,PENDING,,2.5637e-08,0.606991,,,
train_mnist_0abb8_00004,PENDING,,7.35795e-07,0.710213,,,
train_mnist_0abb8_00005,PENDING,,0.000168464,0.310138,,,
train_mnist_0abb8_00006,PENDING,,0.429744,0.144786,,,
train_mnist_0abb8_00007,PENDING,,0.000358131,0.690782,,,
train_mnist_0abb8_00008,PENDING,,9.75779e-10,0.227704,,,
train_mnist_0abb8_00009,PENDING,,2.36833e-07,0.257793,,,


Result for train_mnist_0abb8_00000:
  date: 2022-06-07_22-49-38
  done: true
  experiment_id: bb3a058f213f4036827c15c4f63fc729
  experiment_tag: 0_lr=0.007154,momentum=0.43962
  hostname: DESKTOP-1OR65MK
  iterations_since_restore: 10
  mean_accuracy: 0.74375
  node_ip: 127.0.0.1
  pid: 28424
  time_since_restore: 2.263150930404663
  time_this_iter_s: 0.2127552032470703
  time_total_s: 2.263150930404663
  timestamp: 1654613378
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 0abb8_00000
  warmup_time: 0.004000425338745117
  


...

(pid=) 2022-06-07 22:50:54,870	INFO context.py:67 -- Exec'ing worker with command: "c:\Anaconda3\envs\transformers\python.exe" c:\Anaconda3\envs\transformers\lib\site-packages\ray\workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=1306 --object-store-name=tcp://127.0.0.1:58547 --raylet-name=tcp://127.0.0.1:34295 --redis-address=None --storage=None --temp-dir=C:\Users\tzh\AppData\Local\Temp\ray --metrics-agent-port=25171 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:19312 --redis-password=5241590000000000 --startup-token=31 --runtime-env-hash=-1375307434
2022-06-07 22:50:55,629	INFO trial_runner.py:803 -- starting train_mnist_0abb8_00019


Result for train_mnist_0abb8_00019:
  date: 2022-06-07_22-50-58
  done: true
  experiment_id: 5bdfcd24461e4b9c99bb8f3dcd2fb1ac
  hostname: DESKTOP-1OR65MK
  iterations_since_restore: 1
  mean_accuracy: 0.115625
  node_ip: 127.0.0.1
  pid: 38760
  time_since_restore: 0.2569568157196045
  time_this_iter_s: 0.2569568157196045
  time_total_s: 0.2569568157196045
  timestamp: 1654613458
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 0abb8_00019
  warmup_time: 0.0040433406829833984
  


Trial name,status,loc,lr,momentum,acc,iter,total time (s)
train_mnist_0abb8_00000,TERMINATED,127.0.0.1:28424,0.00715404,0.439615,0.74375,10,2.26315
train_mnist_0abb8_00001,TERMINATED,127.0.0.1:16716,0.00230887,0.681205,0.078125,1,0.246852
train_mnist_0abb8_00002,TERMINATED,127.0.0.1:27544,1.44449e-08,0.681438,0.065625,1,0.244753
train_mnist_0abb8_00003,TERMINATED,127.0.0.1:15152,2.5637e-08,0.606991,0.1125,4,0.900536
train_mnist_0abb8_00004,TERMINATED,127.0.0.1:31076,7.35795e-07,0.710213,0.071875,1,0.264001
train_mnist_0abb8_00005,TERMINATED,127.0.0.1:33260,0.000168464,0.310138,0.11875,4,0.894039
train_mnist_0abb8_00006,TERMINATED,127.0.0.1:28388,0.429744,0.144786,0.8,10,2.17274
train_mnist_0abb8_00007,TERMINATED,127.0.0.1:38972,0.000358131,0.690782,0.215625,4,0.885158
train_mnist_0abb8_00008,TERMINATED,127.0.0.1:32136,9.75779e-10,0.227704,0.090625,1,0.253804
train_mnist_0abb8_00009,TERMINATED,127.0.0.1:36188,2.36833e-07,0.257793,0.025,1,0.246631


Trial name,status,loc,lr,momentum,acc,iter,total time (s)
train_mnist_0abb8_00000,TERMINATED,127.0.0.1:28424,0.00715404,0.439615,0.74375,10,2.26315
train_mnist_0abb8_00001,TERMINATED,127.0.0.1:16716,0.00230887,0.681205,0.078125,1,0.246852
train_mnist_0abb8_00002,TERMINATED,127.0.0.1:27544,1.44449e-08,0.681438,0.065625,1,0.244753
train_mnist_0abb8_00003,TERMINATED,127.0.0.1:15152,2.5637e-08,0.606991,0.1125,4,0.900536
train_mnist_0abb8_00004,TERMINATED,127.0.0.1:31076,7.35795e-07,0.710213,0.071875,1,0.264001
train_mnist_0abb8_00005,TERMINATED,127.0.0.1:33260,0.000168464,0.310138,0.11875,4,0.894039
train_mnist_0abb8_00006,TERMINATED,127.0.0.1:28388,0.429744,0.144786,0.8,10,2.17274
train_mnist_0abb8_00007,TERMINATED,127.0.0.1:38972,0.000358131,0.690782,0.215625,4,0.885158
train_mnist_0abb8_00008,TERMINATED,127.0.0.1:32136,9.75779e-10,0.227704,0.090625,1,0.253804
train_mnist_0abb8_00009,TERMINATED,127.0.0.1:36188,2.36833e-07,0.257793,0.025,1,0.246631


(pid=) 2022-06-07 22:50:58,800	INFO context.py:67 -- Exec'ing worker with command: "c:\Anaconda3\envs\transformers\python.exe" c:\Anaconda3\envs\transformers\lib\site-packages\ray\workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=1306 --object-store-name=tcp://127.0.0.1:58547 --raylet-name=tcp://127.0.0.1:34295 --redis-address=None --storage=None --temp-dir=C:\Users\tzh\AppData\Local\Temp\ray --metrics-agent-port=25171 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:19312 --redis-password=5241590000000000 --startup-token=32 --runtime-env-hash=-1375307434
2022-06-07 22:50:58,878	INFO tune.py:701 -- Total run time: 86.15 seconds (86.01 seconds for the tuning loop).


In [9]:
analysis.get_best_config(metric="mean_accuracy", mode="max")

{'lr': 0.019213874539757453, 'momentum': 0.807170854238808}

In [10]:
a = analysis.get_best_trial(metric="mean_accuracy", mode="max")

In [11]:
a.config

{'lr': 0.019213874539757453, 'momentum': 0.807170854238808}

In [12]:
a.metric_analysis

{'mean_accuracy': {'max': 0.88125,
  'min': 0.390625,
  'avg': 0.78,
  'last': 0.878125,
  'last-5-avg': 0.8643750000000001,
  'last-10-avg': 0.7799999999999999},
 'time_this_iter_s': {'max': 0.2636396884918213,
  'min': 0.20995736122131348,
  'avg': 0.23047401905059817,
  'last': 0.2180461883544922,
  'last-5-avg': 0.22256250381469728,
  'last-10-avg': 0.23047401905059814},
 'done': {'max': False,
  'min': False,
  'avg': 0.0,
  'last': False,
  'last-5-avg': 0.0,
  'last-10-avg': 0.0},
 'training_iteration': {'max': 10,
  'min': 1,
  'avg': 5.5,
  'last': 10,
  'last-5-avg': 8.0,
  'last-10-avg': 5.5},
 'time_total_s': {'max': 2.3047401905059814,
  'min': 0.2636396884918213,
  'avg': 1.3029854774475096,
  'last': 2.3047401905059814,
  'last-5-avg': 1.8630433082580566,
  'last-10-avg': 1.3029854774475098},
 'time_since_restore': {'max': 2.3047401905059814,
  'min': 0.2636396884918213,
  'avg': 1.3029854774475096,
  'last': 2.3047401905059814,
  'last-5-avg': 1.8630433082580566,
  'las

In [13]:
analysis.dataframe(metric="mean_accuracy", mode="max")

,mean_accuracy,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,...,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,warmup_time,config/lr,config/momentum,logdir
0,0.750000,0.227043,False,NaN,NaN,9,0abb8_00000,bb3a058f213f4036827c15c4f63fc729,2022-06-07_22-49-37,1654613377,...,28424,DESKTOP-1OR65MK,127.0.0.1,2.050396,0,9,0.004000,7.154035e-03,0.439615,d:\code\github\data_process\use_transformers\r...
1,0.078125,0.246852,True,NaN,NaN,1,0abb8_00001,212e5488bed946999495244904658f65,2022-06-07_22-49-42,1654613382,...,16716,DESKTOP-1OR65MK,127.0.0.1,0.246852,0,1,0.004004,2.308866e-03,0.681205,d:\code\github\data_process\use_transformers\r...
2,0.065625,0.244753,True,NaN,NaN,1,0abb8_00002,d488c355a28a499995666629ee80a645,2022-06-07_22-49-46,1654613386,...,27544,DESKTOP-1OR65MK,127.0.0.1,0.244753,0,1,0.004000,1.444486e-08,0.681438,d:\code\github\data_process\use_transformers\r...
3,0.131250,0.210145,False,NaN,NaN,3,0abb8_00003,214b29730dcd47f5b59d8250f3b7049b,2022-06-07_22-49-50,1654613390,...,15152,DESKTOP-1OR65MK,127.0.0.1,0.670639,0,3,0.004000,2.563700e-08,0.606991,d:\code\github\data_process\use_transformers\r...
4,0.071875,0.264001,True,NaN,NaN,1,0abb8_00004,bf5ff41899ed4602bd7633445d00efe2,2022-06-07_22-49-54,1654613394,...,31076,DESKTOP-1OR65MK,127.0.0.1,0.264001,0,1,0.004002,7.357946e-07,0.710213,d:\code\github\data_process\use_transformers\r...
5,0.118750,0.230129,False,NaN,NaN,3,0abb8_00005,2b67eefffd264431b87daa964c5d6785,2022-06-07_22-49-58,1654613398,...,33260,DESKTOP-1OR65MK,127.0.0.1,0.682528,0,3,0.003998,1.684643e-04,0.310138,d:\code\github\data_process\use_transformers\r...
6,0.840625,0.210732,False,NaN,NaN,8,0abb8_00006,a1b5d041ea88439498f983207e1f5a40,2022-06-07_22-50-03,1654613403,...,28388,DESKTOP-1OR65MK,127.0.0.1,1.763557,0,8,0.004955,4.297439e-01,0.144786,d:\code\github\data_process\use_transformers\r...
7,0.215625,0.207794,True,NaN,NaN,4,0abb8_00007,cccf5b2e578d45bebae2eb8d94357bc3,2022-06-07_22-50-08,1654613408,...,38972,DESKTOP-1OR65MK,127.0.0.1,0.885158,0,4,0.005008,3.581306e-04,0.690782,d:\code\github\data_process\use_transformers\r...
8,0.090625,0.253804,True,NaN,NaN,1,0abb8_00008,a58c62fca65e46d9840d2a7400055333,2022-06-07_22-50-12,1654613412,...,32136,DESKTOP-1OR65MK,127.0.0.1,0.253804,0,1,0.003957,9.757794e-10,0.227704,d:\code\github\data_process\use_transformers\r...
9,0.025000,0.246631,True,NaN,NaN,1,0abb8_00009,9d340bab03fb4da6984b94f5575af0a9,2022-06-07_22-50-16,1654613416,...,36188,DESKTOP-1OR65MK,127.0.0.1,0.246631,0,1,0.003998,2.368333e-07,0.257793,d:\code\github\data_process\use_transformers\r...


In [20]:
list(analysis.trial_dataframes.keys())

['d:\\code\\github\\data_process\\use_transformers\\ray_train_results\\train_mnist_2022-06-07_22-49-32\\train_mnist_0abb8_00000_0_lr=0.007154,momentum=0.43962_2022-06-07_22-49-32',
 'd:\\code\\github\\data_process\\use_transformers\\ray_train_results\\train_mnist_2022-06-07_22-49-32\\train_mnist_0abb8_00001_1_lr=0.0023089,momentum=0.6812_2022-06-07_22-49-38',
 'd:\\code\\github\\data_process\\use_transformers\\ray_train_results\\train_mnist_2022-06-07_22-49-32\\train_mnist_0abb8_00002_2_lr=1.4445e-08,momentum=0.68144_2022-06-07_22-49-43',
 'd:\\code\\github\\data_process\\use_transformers\\ray_train_results\\train_mnist_2022-06-07_22-49-32\\train_mnist_0abb8_00003_3_lr=2.5637e-08,momentum=0.60699_2022-06-07_22-49-47',
 'd:\\code\\github\\data_process\\use_transformers\\ray_train_results\\train_mnist_2022-06-07_22-49-32\\train_mnist_0abb8_00004_4_lr=7.3579e-07,momentum=0.71021_2022-06-07_22-49-51',
 'd:\\code\\github\\data_process\\use_transformers\\ray_train_results\\train_mnist_2022-0

In [16]:
analysis.results_df

20

In [14]:
# # Plot by epoch
# ax = None  # This plots everything on the same plot
# for d in dfs.values():
#     ax = d.mean_accuracy.plot(ax=ax, legend=False)